In [ ]:
## Client Side
#### Build a protocol class that takes file name and i/o as input from app
### Case 1 : Upload
#### 1. Call upload method of class.
#### 2. Inside upload method - 
##### 2.1 Calculate size of file and divide into equi-sized data chunks
##### 2.2 Initialize requirements for uploading - Buffer window, Ip address, Port 420 (fixed), ADD 
##### 2.3 Encapsulate all the data chunks with headers to convert into 278 bytes sized packets
###### 2.3.1 Transfer as many as possible to the sender window to start upload. 
###### 2.3.2 Keep track of what remains to be sent
###### 2.3.3 When the least seq number packet will be sent, new packet is loaded at end of buffer
##### 2.4 Move through this window based on selective repeat paradigm.
##### 2.5 Send the packets by calling a UDP socket communication function/method - 
###### 2.5.1 Input is 278 byte sized packet, Destination port, IP
###### 2.5.2 Connection establishment will be done before data send.
###### 2.5.3 If not estab, call error
##### 2.6 DO NOT Discriminate between the different types of packets. Just deliver.
##### 2.7 While uploading, we need to receive acks and errors from server.
###### 2.7.1 Therefore, there must be an always running recv function as well that delivers packet to app layer
###### 2.7.2 This packet received is to parsed to check the headers for type of packet and actions are taken.
###### 2.7.3 For an error packet, retransmissions will take place by calling a retransmit function
###### 2.7.4 The retransmit is just another version of upload for single packet.
###### 2.7.5 The ACK for this packet is also recieved by the global recv function.
##### 2.8 As and when ACKs are recived for a particular packet defined by its sequence number in the buffer - 
###### 2.8.1 Check seq num and if lowest then move window forward, call for next packet
###### 2.8.2 Otherwise keep acking packets with a map (or something)
##### 2.9 When buffer is empty and no errors have occurred, terminate connection and return.
#### 3. Wait for response from upload
##### 3.1 Retry if fail
##### 3.2 Return success otherwise


### Case 2 : Download 

In [21]:
import time
import os 
import sys
import socket
import math

#helper
def megabytes_to_bytes(mb):
    return mb * 1024 * 1024

#return packet
def buildPacket(msg,opcode,seq_num,fname):
    print("start packet build\n")

#receive ack/reply from server and return
def recvreply(s):
    while True:
        try:
            ClientData, clientAddr = s.recvfrom(4096)
            break
        except ConnectionResetError:
            print("Error. Port numbers not matching. Exiting. Next time enter same port numbers.")
            continue
        except:
            print("Timeout or some other error")
            continue
    text = ClientData.decode('utf8')
    return text,clientAddr;

#three way shaker
def twhs(s,host,port,fname):
    print("inside twhs")
    msg = "hello connect"
    # buildPacket(msg,opcode,seq_num)
    msg = msg.encode('utf-8')
    s.sendto(msg, (host, port))
    text, clientAddr = recvreply(s)
    print("first reply from twhs"+" "+text)
    rep_split = text.split()
    if rep_split[0]=='denied':
        return False
    msg =  "upload "+ " " +fname
    msg = msg.encode('utf-8')
    s.sendto(msg, (host, port))
    return True

#def uploading(fname,s,host,port):
    ## 100 pkt
    ## mp[0-100]=0
    ## 0 1 2 3 4 
    ## 0A 1 2 3 4
    # buffer
    # 5 pkt ka
    # 0 1 2 3 4
    # 0 ACKED
    # x 1 2 3 4
    # 1 2 3 4 5
    # map[0] = 0

class sender:
    fname = ''
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    host = ''
    port = 0
    def __init__(self,file):
        self.fname = file
        self.host = '127.0.0.1'
        self.port = 5000
        try:
            self.s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
            print("Client socket initialized")
            self.s.setblocking(0)
            self.s.settimeout(15)
        except socket.error:
            print("Failed to create socket")
            sys.exit()
    
    def send(self):
        ## 3 way handshake
        host = self.host
        port = self.port
        s = self.s
        fname = self.fname
        shake = twhs(s,host,port,fname)
        if shake==False:
            sys.exit()
        ## handshake hogaya successfull
        print("Now we shall start sending data.")
        ## send data
        size = os.stat(fname)
        sizeS = size.st_size
        print("File size in bytes: " + str(sizeS))
        Num = int(sizeS/256)
        Num = Num + 1
        print("Number of packets to be sent: " + str(Num))
        till = str(Num)
        tillC = till.encode('utf8')
        s.sendto(tillC, (host,port))
        tillIC = int(Num)
        GetRun = open(fname, "rb")
        c = 0
        while tillIC != 0:
            Run = GetRun.read(256)
            s.sendto(Run, (host,port))
            c += 1
            tillIC -= 1
            print("Packet number:" + str(c))
            print("Data sending in process:")
        

fname = './apples.jpg'
s1 = sender(fname)
s1.send()


Client socket initialized
inside twhs
first reply from twhs okay I am ready
Now we shall start sending data.
File size in bytes: 33333
Number of packets to be sent: 131
Packet number:1
Data sending in process:
Packet number:2
Data sending in process:
Packet number:3
Data sending in process:
Packet number:4
Data sending in process:
Packet number:5
Data sending in process:
Packet number:6
Data sending in process:
Packet number:7
Data sending in process:
Packet number:8
Data sending in process:
Packet number:9
Data sending in process:
Packet number:10
Data sending in process:
Packet number:11
Data sending in process:
Packet number:12
Data sending in process:
Packet number:13
Data sending in process:
Packet number:14
Data sending in process:
Packet number:15
Data sending in process:
Packet number:16
Data sending in process:
Packet number:17
Data sending in process:
Packet number:18
Data sending in process:
Packet number:19
Data sending in process:
Packet number:20
Data sending in process:
